In [92]:
import pandas as pd
import numpy as np
import ydata_profiling as yp
import datetime as dt
import sklearn
from tqdm.notebook import trange

In [93]:
df_train = pd.read_csv("sales_history.csv")
df_test = pd.read_csv("test_origin.csv", index_col=0)

In [94]:
df_test

,商品ID,店舗ID
index,,
0,1000001,0
1,1000001,1
2,1000001,2
3,1000001,3
4,1000001,4
...,...,...
3055,3500001,13
3056,3500001,14
3057,3500001,15


In [95]:
df_test

,商品ID,店舗ID
index,,
0,1000001,0
1,1000001,1
2,1000001,2
3,1000001,3
4,1000001,4
...,...,...
3055,3500001,13
3056,3500001,14
3057,3500001,15


In [96]:
# 予測する商品IDをテストデータから取得
items_name = df_test.iloc[:, 0].unique()

# 予測する商品IDが含まれる行を抽出
df_train = df_train[df_train.iloc[:, 2].isin(items_name)]

In [97]:
# 日付をDateTime型に変換
def ToDate(row):
    
    row.iloc[0] = dt.datetime.strptime(row.iloc[0], '%Y-%m-%d')
    row.iloc[0] = row.iloc[0].date()

    return row


df_train = df_train.apply(ToDate, axis=1)

In [98]:
# 売上個数をintに変換
df_train["売上個数"] = df_train["売上個数"].astype('int')

In [99]:
df_train

,日付,店舗ID,商品ID,商品価格,売上個数
0,2018-01-01,9,1000001,420,1
1,2018-01-01,6,1000001,420,1
2,2018-01-01,10,1000001,420,1
364,2018-01-01,0,1200007,250,1
365,2018-01-01,15,1200007,130,1
...,...,...,...,...,...
1119551,2019-10-31,0,3200026,850,1
1119553,2019-10-31,10,3300001,2040,1
1119562,2019-10-31,15,3400002,1440,1
1119563,2019-10-31,17,3400003,1190,1


In [100]:
df_train.iloc[0, 0].month

1

In [101]:
# "YearMonth"列を追加
df_train.insert(1, "Months", 0)

t1 = dt.date(year=2018, month=1, day=1)
df_train.iloc[5000, 0]


def AddMonths(row):

    months = abs(row[0].year - t1.year)*12 + (row[0].month - t1.month)
    row.iloc[1] = int(months)
    return row

df_train = df_train.apply(AddMonths, axis=1)


In [102]:
df_train

,日付,Months,店舗ID,商品ID,商品価格,売上個数
0,2018-01-01,0,9,1000001,420,1
1,2018-01-01,0,6,1000001,420,1
2,2018-01-01,0,10,1000001,420,1
364,2018-01-01,0,0,1200007,250,1
365,2018-01-01,0,15,1200007,130,1
...,...,...,...,...,...,...
1119551,2019-10-31,21,0,3200026,850,1
1119553,2019-10-31,21,10,3300001,2040,1
1119562,2019-10-31,21,15,3400002,1440,1
1119563,2019-10-31,21,17,3400003,1190,1


In [109]:
# 月・店舗・商品ごとに売上個数を合算

list_months_shop_id_sum = []

for m in trange(22):
    for s in trange(18):
        for i in items_name:
            list = []
            sum = df_train.loc[(df_train["Months"]==m) & (df_train["店舗ID"]==s) & (df_train["商品ID"]==i), "売上個数"].sum()
            list.append(m)
            list.append(i)
            list.append(s)
            list.append(sum)
            list_months_shop_id_sum.append(list)

  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

...

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

In [111]:
list_months_shop_id_sum

[[0, 1000001, 0, 6],
 [0, 1000002, 0, 2],
 [0, 1000003, 0, 0],
 [0, 1000004, 0, 1],
 [0, 1000005, 0, 8],
 [0, 1000006, 0, 2],
 [0, 1000007, 0, 1],
 [0, 1000008, 0, 4],
 [0, 1000009, 0, 1],
 [0, 1000010, 0, 0],
 [0, 1000011, 0, 0],
 [0, 1002291, 0, 0],
 [0, 1002377, 0, 0],
 [0, 1002503, 0, 0],
 [0, 1002582, 0, 0],
 [0, 1100001, 0, 0],
 [0, 1100002, 0, 0],
 [0, 1200001, 0, 1],
 [0, 1200002, 0, 1],
 [0, 1200003, 0, 2],
 [0, 1200004, 0, 4],
 [0, 1200005, 0, 0],
 [0, 1200006, 0, 0],
 [0, 1200007, 0, 5],
 [0, 1200008, 0, 2],
 [0, 1200009, 0, 1],
 [0, 1200010, 0, 2],
 [0, 1200011, 0, 0],
 [0, 1200012, 0, 1],
 [0, 1200013, 0, 1],
 [0, 1200014, 0, 1],
 [0, 1200015, 0, 1],
 [0, 1200016, 0, 0],
 [0, 1200017, 0, 0],
 [0, 1200018, 0, 1],
 [0, 1200019, 0, 1],
 [0, 1200020, 0, 1],
 [0, 1200021, 0, 3],
 [0, 1200022, 0, 3],
 [0, 1200023, 0, 1],
 [0, 1200024, 0, 2],
 [0, 1200025, 0, 1],
 [0, 1200026, 0, 0],
 [0, 1200027, 0, 2],
 [0, 1200028, 0, 1],
 [0, 1200029, 0, 1],
 [0, 1200030, 0, 3],
 [0, 1200031,

In [112]:
# 月・店舗・商品ごと売上個数で新しいデータフレームを作成

df_train_new = pd.DataFrame(list_months_shop_id_sum,
                            columns=["Months", "商品ID", "店舗ID", "売上個数"]
                            )

In [115]:
df_train_new = df_train_new.reset_index(drop=True)

(67320, 4)